# Distribución Metros Cuadrados Construidos


In [12]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and "alfa" in nombre and "pachuca" in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_alfa_abril_2024_pachuca',
 'df_alfa_agosto_2024_pachuca',
 'df_alfa_febrero_2024_pachuca',
 'df_alfa_julio_2024_pachuca',
 'df_alfa_junio_2024_pachuca',
 'df_alfa_marzo_2024_pachuca',
 'df_alfa_mayo_2024_pachuca',
 'df_alfa_q_feb_2023_pachuca',
 'df_alfa_q_jul_2023_pachuca',
 'df_alfa_q_jun_2023_pachuca',
 'df_alfa_q_mar_2023_pachuca',
 'df_alfa_q_nov_2022_pachuca',
 'df_alfa_q_oct_2022_pachuca',
 'df_alfa_q_sep_2023_pachuca']

In [13]:
df_alfa_junio_2024_pachuca.columns

Index(['id', 'Categoria', 'Precio', 'propiedad', 'metros_total',
       'metros_construido', 'precio_m2_terreno', 'precio_m2_construido',
       'tiempo_de_publicacion', 'Estado', 'Tipo', 'Estacionamientos',
       'Recamaras', 'Banos', 'Medio_banos', 'Banos_Total', 'Seguridad',
       'Colonia', 'CP', 'url'],
      dtype='object')

In [14]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    # Verificar si la columna 'm2_contruido' existe en el DataFrame
    
        # Renombrar la columna
    df.rename(columns={'m2_contruido': 'm2_construido','segmento':'categoria','Categoria':'categoria','metros_construido':'m2_construido','Precio':'precio'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [15]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
dataframes_name = []

for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'categoria','m2_construido','precio']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
     # Componer el nombre del DataFrame en la lista dataframes_name
    dataframes_name.append(nombre_df)


In [16]:
rangos_precio = {
    "E1": (0, 500000),
    "E2": (500000, 750000),
    "E3": (750001, 1000000),
    "D1": (1000001, 1250000),
    "D2": (1250001, 1500000),
    "D3": (1500001, 1750000),
    "C1": (1750001, 2000000),
    "C2": (2000001, 2250000),
    "C3": (2250001, 2500000),
    "B1": (2500001, 2750000),  
    "B2": (2750001, 3000000),  
    "B3": (3000001, 3250000),  
    "A1": (3250001, 3500000),
    "A2": (3500001, 3750000),
    "A3": (3750001, 4000000),
    "S1": (4000001, 6000000),
    "S2": (6000001, 8000000),
    "S3": (8000001, 12000000),
    "L1": (12000001, 14000000),
    "L2": (14000001, 16000000),
    "L3": (16000001, 18000000),
    "L+": (18000001, 22000000),
    "ELITE": (22000001, float('inf'))
}

# Función para asignar la categoría según el precio
def asignar_categoria(precio):
    for categoria, (limite_inferior, limite_superior) in rangos_precio.items():
        if limite_inferior <= precio < limite_superior:
            return categoria
    return None  # En caso de que el precio no caiga en ningún rango (caso raro)

In [17]:
for df in dataframes_list:    
    df = df.drop(columns=['categoria'])
    # Asignar la categoría real a cada registro
    df['categoria'] = df['precio'].apply(asignar_categoria)

# Cantidad de registros por metros cuadrados construidos, por segmento

In [18]:
def procesar_dataframes(dataframes_list, dataframes_name):
    for i, df in enumerate(dataframes_list):
        # Crear una nueva columna con la primera letra de la categoría
        df['segmento'] = df['categoria'].str[0]
        df.drop(columns=['categoria'], inplace=True)

        # Clasificar en rangos específicos
        bins = [0, 100, 150, 200, 250, 300, float('inf')]
        labels = ['0-100', '100-150', '150-200', '200-250', '250-300', '300+']
        df['m2_construido'] = pd.cut(df['m2_construido'], bins=bins, labels=labels, right=False)

        # Agrupar y contar los datos
        tabla = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)

        # Imprimir el nombre del DataFrame y la tabla resultante
        print(f"Nombre del DataFrame: {dataframes_name[i]}")
        print(tabla)

# Procesar los DataFrames de la lista
procesar_dataframes(dataframes_list, dataframes_name)

Nombre del DataFrame: df_alfa_abril_2024_pachuca
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                  3       13       18       73       20     6
B                  7       17       64       89       18     2
C                  3      106       99       27        8     4
D                 94      188       37       13        3     1
E                 58       47       22       44       17    23
L                  0        0        0        0        1     0
S                  0        9       22       64       59   103
Nombre del DataFrame: df_alfa_agosto_2024_pachuca
m2_construido  0-100  100-150  150-200  200-250  250-300  300+
segmento                                                      
A                  1       20       16       66       21     6
B                  2       26       69       72       17     5
C                 22      102       66       17       10     4
D                 7

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2613299638.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2613299638.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2613299638.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

## Tabla general de frecuencia
 Recuento total de filas para cada combinación de valores únicos de 'segmento' y 'm2_contruido' con todos los DataFrame

In [19]:
# Crear una nueva función para sumar las tablas de todos los DataFrames
def sumar_tablas(dataframes_list):
    # Inicializar una tabla vacía para almacenar la suma
    tabla_suma = None
    
    # Iterar sobre cada DataFrame en la lista
    for df in dataframes_list:
        # Agrupar y contar los datos en el DataFrame actual
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        # Sumar la tabla actual al acumulador (tabla_suma)
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    # Imprimir la tabla resultante de la suma
    print("Tabla general de frecuencia:")
    print(tabla_suma)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
sumar_tablas(dataframes_list)

Tabla general de frecuencia:
m2_construido   0-100  100-150  150-200  200-250  250-300    300+
segmento                                                         
A                60.0    250.0    321.0   1242.0    422.0   139.0
B               134.0    488.0   1450.0   1307.0    254.0    72.0
C               282.0   1605.0   1211.0    305.0     99.0    44.0
D              1735.0   1884.0    372.0     96.0     35.0    40.0
E               947.0    313.0    196.0    252.0    128.0   212.0
L                 2.0      0.0      0.0      1.0      8.0   211.0
S                51.0    186.0    323.0    687.0    934.0  2087.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\470711682.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\470711682.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\470711682.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\470711682.py:9: FutureWarning:

The defaul

In [20]:
orden_segmentos=["E", "D", "C", "B","A", "S", "L",'ELITE' ]

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    segmentos = tabla.index
     # Reordenar segmentos según el orden definido
    segmentos_ordenados = [seg for seg in orden_segmentos if seg in segmentos]
    
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, segmento in enumerate(segmentos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=banos,
            y=tabla.loc[segmento],
            name=segmento,
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Rango de M2",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)
# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)

Tabla general de frecuencia:
segmento          A     B     C     D    E      L     S
m2_construido                                          
0-100            60   134   282  1735  947    2.0    51
100-150         250   488  1605  1884  313    0.0   186
150-200         321  1450  1211   372  196    0.0   323
200-250        1242  1307   305    96  252    1.0   687
250-300         422   254    99    35  128    8.0   934
300+            139    72    44    40  212  211.0  2087


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2074870457.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2074870457.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2074870457.py:7: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2074870457.py:7: FutureWarning:

The de

In [21]:
# Definir el orden deseado de los segmentos
orden_segmentos = ["E", "D", "C", "B", "A", "S", "L", 'ELITE']

def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        # Agrupar y contar casas según 'm2_construido' y 'segmento'
        tabla_actual = df.groupby(['segmento', 'm2_construido']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    # Reordenar los índices (segmentos) de acuerdo a 'orden_segmentos'
    tabla_suma = tabla_suma.reindex(orden_segmentos).fillna(0)
    
    # Graficar la tabla reordenada
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):  
    # Obtenemos las columnas (m2_construido) y los índices (segmentos) de la tabla
    segmentos = tabla.index
    m2_construido = tabla.columns
    
    fig = go.Figure()
    
    # Trazar las barras para cada rango de m2_construido dentro de cada segmento
    for i, m2 in enumerate(m2_construido):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=segmentos,            # Segmentos en el eje X
            y=tabla[m2],            # Cantidad de casas para ese rango de m2_construido
            name=f"{m2}",        # Nombre del rango de m2_construido
            marker_color=color
        ))
    
    fig.update_layout(
        xaxis_title="Segmento",        
        yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        legend_title="Rango M2 construido",
        barmode='group',  # Las barras se agrupan por rango de m2_construido
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10)  # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

# Llamar a la función sumar_tablas con la lista de DataFrames como argumento
fig = sumar_tablas(dataframes_list)


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2594576605.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2594576605.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2594576605.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2594576605.py:9: FutureWarning:

The de

Tabla general de frecuencia:
m2_construido   0-100  100-150  150-200  200-250  250-300    300+
segmento                                                         
A                60.0    250.0    321.0   1242.0    422.0   139.0
B               134.0    488.0   1450.0   1307.0    254.0    72.0
C               282.0   1605.0   1211.0    305.0     99.0    44.0
D              1735.0   1884.0    372.0     96.0     35.0    40.0
E               947.0    313.0    196.0    252.0    128.0   212.0
L                 2.0      0.0      0.0      1.0      8.0   211.0
S                51.0    186.0    323.0    687.0    934.0  2087.0


C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2594576605.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\2594576605.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## GRAL

In [22]:
def sumar_tablas(dataframes_list):
    tabla_suma = None
    
    for df in dataframes_list:
        tabla_actual = df.groupby(['m2_construido', 'segmento']).size().unstack(fill_value=0)
        
        if tabla_suma is None:
            tabla_suma = tabla_actual
        else:
            tabla_suma = tabla_suma.add(tabla_actual, fill_value=0)
    
    print("Tabla general de frecuencia:")
    print(tabla_suma)
    
    fig = graficar_tabla(tabla_suma)
    
    # Guardar la gráfica como archivo HTML
    guardar_grafico_como_html(fig, 'g_pie_dist_m2_construidos', carpeta='assets/graficas')
    
    return fig

colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6','#4225CF','#2CBA4D']

def graficar_tabla(tabla):
    # Sumar los valores por 'm2_contruido' para obtener el total por rango de M2
    suma_por_m2 = tabla.sum(axis=1)
    
    fig = go.Figure(data=[go.Pie(
        labels=suma_por_m2.index.astype(str),
        values=suma_por_m2.values,
        marker=dict(colors=colores),
        textinfo='percent',
        insidetextorientation='radial'
    )])
    
    fig.update_layout(
        # title="Distribución de M2 construidos",
        legend_title="Rango M2 Construido",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02, 
            xanchor='right',
            x=0.72
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10) # Ajusta los márgenes (left, right, top, bottom)
    )    
    
    fig.show()
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Guardar la gráfica como archivo HTML en la carpeta especificada
    ruta_archivo = os.path.join(carpeta, f'{nombre_archivo}.html')
    pio.write_html(fig, ruta_archivo)

fig1 = sumar_tablas(dataframes_list)

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\1646451166.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\1646451166.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\1646451166.py:5: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\yoe11\AppData\Local\Temp\ipykernel_1712\1646451166.py:5: FutureWarning:

The de

Tabla general de frecuencia:
segmento          A     B     C     D    E      L     S
m2_construido                                          
0-100            60   134   282  1735  947    2.0    51
100-150         250   488  1605  1884  313    0.0   186
150-200         321  1450  1211   372  196    0.0   323
200-250        1242  1307   305    96  252    1.0   687
250-300         422   254    99    35  128    8.0   934
300+            139    72    44    40  212  211.0  2087


# DIRECTOS

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

data = {
    'Segmento': ['E', 'D', 'C', 'B', 'A', 'S', 'L'],    
    '0-100': [58,90,127,171,118,737,28],    
    '100-150': [13,16,14,31,56,668,90],    
    '150-200': [1,7,5,2,7,212,126],    
    '200-250': [1,2,4,12,0,61,125],    
    '250-300': [4,1,1,0,0,22,102],    
    '>300': [2,11,2,0,2,30,275],        
}
# Convertir a DataFrame
dataframes_list = pd.DataFrame(data)

In [2]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']
orden_segmentos=["E", "D", "C", "B","A", "S", "L"]

def graficar_tabla(tabla):
    segmentos = tabla.index
    banos = tabla.columns
    
    fig = go.Figure()
    
    for i, estacionamiento in enumerate(banos):
        color = colores[i % len(colores)]
        fig.add_trace(go.Bar(
            x=segmentos,
            y=tabla.loc[:, estacionamiento],
            name=estacionamiento,
            marker_color=color
        ))
    
    fig.update_layout(
        #title="M2 de Terreno",
        #xaxis_title="Rango de M2",
         yaxis=dict(
            title='Número de casas',
            gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
            gridwidth=1,
        ),
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        legend_title="Rango M2 construido",
        barmode='group',
        legend=dict(
            orientation='h',  # Orientación horizontal de la leyenda
            yanchor='bottom',  # Anclar al borde inferior
            y=1.02,  # Colocar la leyenda justo debajo del gráfico
            xanchor='right',
            x=1
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función graficar_tabla con la tabla como argumento
data = {
    'Segmento': ['E', 'D', 'C', 'B', 'A', 'S', 'L'],    
    '0-100': [58,90,127,171,118,737,28],    
    '100-150': [13,16,14,31,56,668,90],    
    '150-200': [1,7,5,2,7,212,126],    
    '200-250': [1,2,4,12,0,61,125],    
    '250-300': [4,1,1,0,0,22,102],    
    '>300': [2,11,2,0,2,30,275],        
}
df = pd.DataFrame(data)
df = df.set_index('Segmento')
fig = graficar_tabla(df)
# Guardar la gráfica como archivo HTML
guardar_grafico_como_html(fig, 'g_bar_dist_m2_construidos', carpeta='assets/graficas')

In [3]:
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#F79CB9','#E50CB6','#4225CF']
orden_segmentos=["E", "D", "C", "B","A", "S", "L"]

def graficar_tabla(tabla):
    # Calcular el porcentaje de cada estacionamiento
    porcentajes = tabla.sum() / tabla.sum().sum()
    
    fig = go.Figure(data=[go.Pie(
        labels=tabla.columns,
        values=porcentajes,
        marker=dict(colors=colores[:len(tabla.columns)]),
    )])
    
    fig.update_layout(
        #title="Distribución de Estacionamientos",
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
    )
    
    fig.show()    
    return fig

def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.png')

# Llamar a la función graficar_tabla con la tabla como argumento
data = {
    '0-100': [58,90,127,171,118,737,28],    
    '100-150': [13,16,14,31,56,668,90],    
    '150-200': [1,7,5,2,7,212,126],    
    '200-250': [1,2,4,12,0,61,125],    
    '250-300': [4,1,1,0,0,22,102],    
    '>300': [2,11,2,0,2,30,275],        
}
df = pd.DataFrame(data)
fig = graficar_tabla(df)
# Guardar la gráfica como archivo HTML
guardar_grafico_como_html(fig, 'g_pie_dist_m2_construidos', carpeta='assets/graficas')